In [2]:
from scipy import sparse
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from umap import UMAP

## 1 Загрузка датасета

In [3]:
train = sparse.load_npz("train.npz")
train

<14590x77888 sparse matrix of type '<class 'numpy.float64'>'
	with 2026769 stored elements in Compressed Sparse Row format>

Видим, что матрица разреженная с большим количеством фичей

## 2 Используем метод k-средних

In [4]:
train = sparse.load_npz("train.npz")

kmeans = KMeans(n_clusters=5, random_state=42)

labels = kmeans.fit_predict(train)

df = pd.DataFrame({'ID': range(len(labels)), 'TARGET': labels})

df.to_csv('predictions.csv', index=False) # score -0.02310695083141269

C:\dev\miniconda\envs\ds\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## 3 Используем DBSCAN для получения кластеров

Используем TSNE для снижения размерности, но видим, что модель работает долго

In [3]:
tsne = TSNE(n_components=2)

train_tsne = tsne.fit_transform(train.toarray())

dbscan = DBSCAN()

labels = dbscan.fit_predict(train_tsne)

df = pd.DataFrame({'ID': range(len(labels)), 'TARGET': labels})

df.to_csv('predictions.csv', index=False) # score 0.004331793159908802

Используем TruncatedSVD для снижения размерности, в итоге получаем более быструю модель с аналогичным результатом

In [6]:
svd = TruncatedSVD(n_components=100)
train_svd = svd.fit_transform(train)

dbscan = DBSCAN()
labels = dbscan.fit_predict(train_svd)

df = pd.DataFrame({'ID': range(len(labels)), 'TARGET': labels})

df.to_csv('predictions.csv', index=False) # score 0.004331793159908802

Теперь попробуем использовать для снижения размерности UMAP

In [5]:
u = UMAP(n_components=100)
train_umap = u.fit_transform(train)

dbscan = DBSCAN()
labels = dbscan.fit_predict(train_umap)

df = pd.DataFrame({'ID': range(len(labels)), 'TARGET': labels})

df.to_csv('predictions.csv', index=False) # score 0.025511201922304817

В итоге результат улучшился